from utility import *
from samplers import *
from pool import *

def runMe(sequence_length,num_rounds=3,affinity_max=1e6):
    """
    Simulate a basic binding experiment.
    """
     
    scalar = 20**(sequence_length)
    library_size = scalar*2.8e-5
    
    print(scalar, library_size, 9.77e-10*scalar)
    
    pipet = PipetteSampler()
    screen = BindingSampler(conc_constant=1.5e18)
    amplify = PhageAmplificationSampler()

    pool = Pool(sequence_length=sequence_length,alphabet=AMINO_ACIDS)
    pool.createUniformPool(library_size,affinity_max)

    try:
        print(0,pool.current_counts.size,sum(pool.current_counts))
        for i in range(num_rounds):
            pipet.runExperiment(pool,library_size)  #7e5) #100*scalar)
            screen.runExperiment(pool,1.5e-3*scalar) #4e7) #1*scalar)
            amplify.runExperiment(pool,6.8e-3*scalar,checkpoint=True) #1.75e8) #100*scalar,checkpoint=True)

            print(i+1,pool.current_counts.size,sum(pool.current_counts))
    
    except PoolIsEmptyError:
        pass
    
    
    return pool






In [7]:
import sys
import pool, samplers, utility
from scipy.optimize import minimize

class Exp:
    
    def __init__(self,sequence_length=8,alphabet=pool.AMINO_ACIDS):
        
        self.company_tube_fx =      2.60e-3
        self.pipet_onto_plate_fx =  2.81e-5
        self.protein_on_plate_fx =  1.50e-3
        self.amplify_post_bind_fx = 6.40e-3
        self.illumina_reads_fx    = 2.44e-10
        self.first_round_recov_fx = 9.77e-10

        self.sequence_length = sequence_length
        self.alphabet = alphabet
        
        self.total_possible = len(alphabet)**sequence_length
        
        self.company_tube = int(self.company_tube_fx*self.total_possible)
        self.pipet_onto_plate = int(self.pipet_onto_plate_fx*self.total_possible)
        self.protein_on_plate = int(self.protein_on_plate_fx*self.total_possible)
        self.amplify_post_bind = int(self.amplify_post_bind_fx*self.total_possible)
        self.illumina_reads = int(self.illumina_reads_fx*self.total_possible)
        self.first_round_recov = int(self.first_round_recov_fx*self.total_possible)

    def create(self,affinity_max=1e6,skew=3,tol=1,max_num_reps=100,
               conc_const_guess=10):
                
        self.pipet = samplers.PipetteSampler()
        self.amplify = samplers.PhageAmplificationSampler()

        self.p = pool.Pool(sequence_length=self.sequence_length,alphabet=self.alphabet)
        self.p.createScaledPool(self.pipet_onto_plate,affinity_max,scale_reps=skew)
        
        
        print("Optimizing concentration constant")
        self.conc_constant = minimize(self._objective,conc_const_guess,
                                      method='nelder-mead',
                                      options={'xtol': tol, 'disp': False})
        
        print(self.conc_constant.x)
        

    def _objective(self,conc_const):
        
        screen = samplers.BindingSampler(conc_constant=(10**conc_const))
        self.pipet.runExperiment(self.p,self.pipet_onto_plate)
        recovery = screen.runExperiment(self.p,self.protein_on_plate,
                                        return_only_sample_size=True)
        self.p.reset()
            
        print(conc_const[0],recovery,"vs",self.first_round_recov)
            
        return abs(self.first_round_recov - recovery)
            

e = Exp()
e.create()
        
        

def runMe(sequence_length,num_rounds=3,affinity_max=1e6,alphabet=pool.AMINO_ACIDS,
          pipet_scalar=2.81e-5,screen_scalar=1.5e-3,amplify_scalar=6.4e-3,company_scalar=2.6e-3,
          conc_constant=5e14):
    """
    Simulate a basic binding experiment.  
    """
    
    total_possible = len(alphabet)**(sequence_length)
    library_size = total_possible*company_scalar
    
    pipet_scale = pipet_scalar*total_possible
    protein_on_plate_scale = screen_scalar*total_possible
    amplify_scale = amplify_scalar*total_possible
    
    self.pipet = samplers.PipetteSampler()
    self.screen = samplers.BindingSampler(conc_constant=conc_constant)
    self.amplify = samplers.PhageAmplificationSampler()

    p = pool.Pool(sequence_length=sequence_length,alphabet=alphabet)
    p.createScaledPool(pipet_scale,affinity_max,scale_reps=3)

    
    samples = []
    
    contents, counts = amplify.runExperiment(p,650,append_to_current=False)
    samples.append(list(zip(contents,counts)))
    
    try:
        print(0,p.current_counts.size,sum(p.current_counts))
        sys.stdout.flush()
        for i in range(num_rounds):
            self.pipet.runExperiment(p,pipet_scale)
            self.screen.runExperiment(p,protein_on_plate_scale)
            self.amplify.runExperiment(p,amplify_scale,checkpoint=True)
            
            self.contents, counts = amplify.runExperiment(p,650,append_to_current=False)
            samples.append(list(zip(contents,counts)))

            print(i+1,p.current_counts.size,sum(p.current_counts))
    
    except samplers.PoolIsEmptyError:
        pass

    
    unique_keys = []
    for s in samples:
        unique_keys.extend([k[0] for k in s])
    
    unique_keys = list(dict([(k,()) for k in unique_keys]).keys())
    template  = [0 for i in range(len(samples))]
    out_dict = {}
    for u in unique_keys:
        out_dict[u] = template[:]
        
    for i, s in enumerate(samples):
        for c in s:
            out_dict[c[0]][i] = c[1]
        
    return out_dict
    
    
    #return p


Optimizing concentration constant
10.0 1297225 vs 25
10.5 419918 vs 25
11.0 133790 vs 25
11.5 42409 vs 25
12.5 4245 vs 25
13.5 425 vs 25
15.5 4 vs 25
17.5 0 vs 25
17.5 0 vs 25
16.5 0 vs 25
14.5 42 vs 25
13.5 425 vs 25
13.5 425 vs 25
15.0 13 vs 25
15.5 4 vs 25
14.75 24 vs 25
14.5 42 vs 25
14.875 18 vs 25
14.625 32 vs 25
14.8125 21 vs 25
14.6875 28 vs 25
14.71875 26 vs 25
[ 14.75]


In [ ]:
p = runMe(8,num_rounds=3,affinity_max=1e6)


import pickle
f = open("yo.pickle","wb")
pickle.dump(p,f)
#pickle.dump(p.round_count_dict,f)
f.close()


import matplotlib.pyplot as plt
%matplotlib inline

pool.round_affinities(0)

m = len(pool.checkpoints)
for i, x in enumerate(pool.checkpoints):
    if x == True:
        plt.plot(pool.round_affinities(i),pool.round_counts(i)/np.sum(pool.round_counts(i)),"o",color=[1-i/m,1-i/m,i/m])


unique_contents = []
for i, x in enumerate(pool.checkpoints):
    if x == True:
        unique_contents.extend(pool.round_contents(i))
        
counter = 0
out_dict = dict([(k,[0,0,0,0,0]) for k in unique_contents])
for i, x in enumerate(pool.checkpoints):
    if x == True:        
        counts = pool.round_counts(i)
        affinities = pool.round_affinities(i)
        
        for j, x in enumerate(pool.round_contents(i)):            
            out_dict[x][counter] = counts[j]
            out_dict[x][4] = affinities[j]
        
        counter = counter + 1
    
final_dict = {}
for k in out_dict.keys():
    if out_dict[k][1] != 0:
        final_dict[k] = out_dict[k][:]

plt.xscale('log')
plt.yscale('log')
plt.show()
    

out = []
for k in final_dict.keys():
    x = final_dict[k]
    out.append((final_dict[k][4],final_dict[k]))

out.sort()

for a in out:
    plt.plot([1,2,3],a[1][1:4])
    #plt.plot([1,2,3],a[1][1:4],"o")
    
plt.show()